In [6]:
import pandas as pd
import seaborn as sns

In [7]:
injuryData = pd.read_csv("~/Documents/Job Search - 2023/NBA_Injury_Predictor/Injuries.csv")

In [8]:
injuryData.rename(columns={'Unnamed: 0': 'ID'}, inplace=True)

In [9]:
injuryData

,ID,Date,Team,Acquired,Relinquished,Notes
0,0,1951-12-25,Bullets,NaN,Don Barksdale,placed on IL
1,1,1952-12-26,Knicks,NaN,Max Zaslofsky,placed on IL with torn side muscle
2,2,1956-12-29,Knicks,NaN,Jim Baechtold,placed on inactive list
3,3,1959-01-16,Lakers,NaN,Elgin Baylor,player refused to play after being denied a ro...
4,4,1961-11-26,Lakers,NaN,Elgin Baylor,player reported for military duty
...,...,...,...,...,...,...
37662,37662,2023-04-16,Clippers,Marcus Morris,NaN,activated from IL
37663,37663,2023-04-16,Grizzlies,Dillon Brooks,NaN,activated from IL
37664,37664,2023-04-16,Grizzlies,Ja Morant,NaN,activated from IL
37665,37665,2023-04-16,Grizzlies,Jaren Jackson Jr.,NaN,activated from IL


### `We want to separate the dataset by decade to get a good idea of the total number of injuries that have occured decade by decade`

In [12]:
# Setting the objects in the column to a pandas datetime object
injuryData['Date'] = pd.to_datetime(injuryData['Date'])

#Grouping rows by 'decade' column
injuryData['Decade'] = (injuryData['Date'].dt.year // 10) *10

In [13]:
injuryData

,ID,Date,Team,Acquired,Relinquished,Notes,Decade
0,0,1951-12-25,Bullets,NaN,Don Barksdale,placed on IL,1950
1,1,1952-12-26,Knicks,NaN,Max Zaslofsky,placed on IL with torn side muscle,1950
2,2,1956-12-29,Knicks,NaN,Jim Baechtold,placed on inactive list,1950
3,3,1959-01-16,Lakers,NaN,Elgin Baylor,player refused to play after being denied a ro...,1950
4,4,1961-11-26,Lakers,NaN,Elgin Baylor,player reported for military duty,1960
...,...,...,...,...,...,...,...
37662,37662,2023-04-16,Clippers,Marcus Morris,NaN,activated from IL,2020
37663,37663,2023-04-16,Grizzlies,Dillon Brooks,NaN,activated from IL,2020
37664,37664,2023-04-16,Grizzlies,Ja Morant,NaN,activated from IL,2020
37665,37665,2023-04-16,Grizzlies,Jaren Jackson Jr.,NaN,activated from IL,2020


`Now let's calculate the total sum of injuries for each decade`

In [24]:

for decade, group in injuryData.groupby('Decade'):
    print(f"Decade: {decade} Injuries: {group.shape[0]}")
    print("---------------------------------------------")

Decade: 1950 Injuries: 4
---------------------------------------------
Decade: 1960 Injuries: 10
---------------------------------------------
Decade: 1970 Injuries: 167
---------------------------------------------
Decade: 1980 Injuries: 547
---------------------------------------------
Decade: 1990 Injuries: 3168
---------------------------------------------
Decade: 2000 Injuries: 10773
---------------------------------------------
Decade: 2010 Injuries: 15658
---------------------------------------------
Decade: 2020 Injuries: 7340
---------------------------------------------
